# Yarkoni Figure

In [ ]:
import numpy as np
import pylab as plt
import seaborn as sns
from scipy.special import gammaln
plt.ion()
sns.set_style(style='white')
sns.set_context("talk")

def spm_hrf(RT, P=None, fMRI_T=16):
    p = np.array([6, 16, 1, 1, 6, 0, 32], dtype=float)
    if P is not None:
        p[0:len(P)] = P

    _spm_Gpdf = lambda x, h, l: np.exp(h * np.log(l) + (h - 1) * np.log(x) - (l * x) - gammaln(h))
    # modelled hemodynamic response function - {mixture of Gammas}
    dt = RT / float(fMRI_T)
    u = np.arange(0, int(p[6] / dt + 1)) - p[5] / dt
    with np.errstate(divide='ignore'):  # Known division-by-zero
        hrf = _spm_Gpdf(u, p[0] / p[2], dt / p[2]) - _spm_Gpdf(u, p[1] / p[3],
                                                               dt / p[3]) / p[4]
    idx = np.arange(0, int((p[6] / RT) + 1)) * fMRI_T
    hrf = hrf[idx]
    hrf = hrf / np.sum(hrf)
    return hrf

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Generate sample data.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Initialize HRF.
sfreq = 1e-1
hrf = spm_hrf(sfreq)

## Construct boxcars.
times = np.arange(0,2+sfreq,sfreq)
boxcars = np.zeros((4,times.shape[0]))

mask = (times >= 0.05) & (times <= 1)
boxcars[0,mask] += 1.00    # Low conflict
boxcars[1,mask] += 1.25    # High conflict
boxcars[2,mask] += 1.00    # Fast RT
mask = (times >= 0.05) & (times <= 1.25)
boxcars[3,mask] += 1.00    # Slow RT

## Construct BOLDF response.
BOLD = np.apply_along_axis(np.convolve, 1, boxcars, hrf)
bold_times = np.arange(0,BOLD.shape[-1]*sfreq,sfreq)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Plotting.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

fig = plt.figure(figsize=(12,6))
colors = ['#377eb8','#4daf4a']
lw = 2.5

## Amplitude differences.
ax = plt.subplot2grid((2,3),(0,0))
for n, color in zip([0,1],colors):
    ax.plot(times, boxcars[n], lw=lw, color=color, alpha=0.7)
ax.hlines(0,-0.1,2,linestyle='--',color='k',alpha=0.4)
ax.set_xlim(-0.1,2)
ax.set_ylim(-0.1,1.5)
ax.set_yticks([])
ax.set_xlabel('Time (s)', fontsize=16)
ax.set_title('A. Differences in amplitude of physiological processes', 
             loc='left', weight='bold', fontsize=18)

ax = plt.subplot2grid((2,3),(0,1),colspan=2)
for n, color, label in zip([0,1],colors,['Low Effort','High Effort']):
    ax.plot(bold_times, BOLD[n], lw=lw, color=color, alpha=0.7, label=label)
ax.hlines(0,-0.1,16,linestyle='--',color='k',alpha=0.4)
ax.set_xlim(-0.1,16)
ax.set_xticks([0,5,10,15])
ax.set_ylim(-0.02,0.3)
ax.set_yticks([])
ax.set_xlabel('Time (s)', fontsize=16)
ax.legend(loc=1, frameon=False, fontsize=14)

## Duration differences.
ax = plt.subplot2grid((2,3),(1,0))
for n, color in zip([2,3],colors):
    ax.plot(times, boxcars[n], lw=lw, color=color, alpha=0.7)
ax.hlines(0,-0.1,2,linestyle='--',color='k',alpha=0.4)
ax.set_xlim(-0.1,2)
ax.set_ylim(-0.1,1.5)
ax.set_yticks([])
ax.set_xlabel('Time (s)', fontsize=16)
ax.set_title('B. Differences in duration of physiological processes', 
             loc='left', weight='bold', fontsize=18)


ax = plt.subplot2grid((2,3),(1,1),colspan=2)
for n, color, label in zip([2,3],colors,['Fast RT','Long RT']):
    ax.plot(bold_times, BOLD[n], lw=lw, color=color, alpha=0.7, label=label)
ax.hlines(0,-0.1,16,linestyle='--',color='k',alpha=0.4)
ax.set_xlim(-0.1,16)
ax.set_xlim(0,16)
ax.set_xticks([0,5,10,15])
ax.set_ylim(-0.02,0.3)
ax.set_yticks([])
ax.set_xlabel('Time (s)', fontsize=16)
ax.legend(loc=1, frameon=False, fontsize=14)

plt.subplots_adjust(left=0.02, right=0.98, bottom=0.125, hspace=0.6)
# plt.show()
plt.savefig('plots/FINAL/yarkoni_plot.png', dpi=120)
plt.close('all')

# Behavior Supplementary Materials

In [ ]:
import cPickle, os
import numpy as np
from collections import defaultdict
from pandas import DataFrame, concat, read_csv

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## I/O parameters.
results_file = 'arc_hierarchical_add_FINAL'

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Load data.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Open results file.
print 'Loading data.'
f = '%s.pickle' %results_file
with open(f, 'rb') as f: results = cPickle.load(f)

## Open model diagnostic file.
info = read_csv('arc_hierarchical_add_FINAL.csv').set_index('Unnamed: 0')

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Prepare model diagnostics.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

print 'Preparing data.'

## Restrict to columns of interest.
info = info[info.columns[3:]]

## Restrict to parameters of interest.
params = np.concatenate([['beta[%s,%s]' %(n,m) for m in range(4)] +\
                         ['a%s[%s]' %(m,n) for m in range(2)] 
                         for n in range(28)])
params = np.concatenate([params, ['beta_mu[%s]' %m for m in range(4)] + ['a%s_mu' %m for m in range(2)]])
info = info.ix[params]

## Append subject / parameter info.
info['Subject'] = np.repeat(np.append(results['Subjects'], 'Group'),6)
info['Param'] = [s[:2] if s.startswith('a') else 'b%s' %s[-2] for s in info.index]
info = info.sort_values(['Subject','Param'])

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Re-standardize coefficients
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
## See Kruschke, 2011 [pgs. 424, 566]
print 'Standardizing coefficients.'
X = results['X']

for col in info.columns[:5]:
    
    ## Extract information.
    b0 = info.loc[info.Param=='b0',col].as_matrix()
    b3 = info.loc[info.Param=='b3',col].as_matrix()
    a1 = info.loc[info.Param=='a1',col].as_matrix()
    
    ## Transform the intercept (b0).    
    info.loc[info.Param=='b0',col] = b0 - (b3 * np.mean(X[:,-1]) / np.std(X[:,-1]))
        
    ## Transform the reward coefficient (b3).
    info.loc[info.Param=='b3',col] = b3 / np.std(X[:,-1]) 
    
    ## Transformed conflict coefficient (a1).
    info.loc[info.Param=='a1',col] = a1 / 4.
    
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Save.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

info = info[info.columns[-2:].tolist() + info.columns[:-2].tolist()]
info.to_csv('arc_hierarchical_add_FINAL_supp_table.csv', index=False)
print 'Done.'

# Behavior Manipulation Check

In [ ]:
import numpy as np
from pandas import read_csv

info = read_csv('stan_results/arc_hierarchical_add_FINAL_supp_table.csv')
subjects = info.Subject.unique()[1:]

for param in ['b1', 'b2', 'b3', 'a1']: 
    
    ix = np.in1d(info.Subject, subjects)&(info.Param==param)
    sign = np.sign(info.loc[ix, '2.5%'])
    zero = np.sign(info.loc[ix, '2.5%']) * np.sign(info.loc[ix, '97.5%'])
    pool = np.where(zero < 0, 0, np.where(sign > 0, 1, -1))
    print param, np.unique(pool, return_counts=True)

# Behavior Histograms (Likelihood of Take / Reaction Time)

In [ ]:
import cPickle, os
import numpy as np
import pylab as plt
import seaborn as sns
sns.set_style(style='white')
sns.set_context("talk")
plt.ion()

def logistic(arr): return 1. / (1 + np.exp(-arr))

def hdi_mask(arr,p):
    lb = np.percentile(arr, (100-p)/2.)
    ub = np.percentile(arr, p+(100-p)/2.)
    return np.where(arr < lb, np.nan, np.where(arr > ub, np.nan, arr))

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## I/O parameters.
results_file = 'arc_hierarchical_add_FINAL'

## Sampling parameters.
n_reward = 1000
hdi = 95.

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Load data.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Open results file.
print 'Loading data.'
f = 'stan_results/%s.pickle' %results_file
with open(f, 'rb') as f: results = cPickle.load(f)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Re-standardize coefficients
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
## See Kruschke, 2011 [pgs. 424, 566]

## Make a copy.
X = results['X']
Beta = results['beta'].copy()
Beta_mu = results['beta_mu'].copy()

## Transform the intercept coefficient.
Beta[:,:,0] = Beta[:,:,0] - (Beta[:,:,-1] * np.mean(X[:,-1]) / np.std(X[:,-1]))    
Beta_mu[:,0] = Beta_mu[:,0] - (Beta_mu[:,-1] * np.mean(X[:,-1]) / np.std(X[:,-1]))    

## Transform the reward coefficient.
Beta[:,:,-1] = Beta[:,:,-1] / np.std(X[:,-1]) 
Beta_mu[:,-1] = Beta_mu[:,-1] / np.std(X[:,-1]) 

## Append.
# results['Beta'] = Beta
# results['Beta_mu'] = Beta_mu

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Compute AUC.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
print 'Computing AUC.'

## Get subjects list. Preallocate space.
n_samp, n_subj, n_pred = Beta.shape
AUC = np.zeros((n_samp, n_subj))

## Compute AUC across subjects.
reward = np.linspace(0,1,n_reward)
for n in range(n_subj):
    
    for m in range(n_pred-1):
        
        AUC[:,n] += logistic(Beta[:,n,:m+1].sum(axis=-1) + np.outer(Beta[:,n,-1], reward)).mean(axis=1)
        
AUC /= n_pred - 1.

## Compute AUC for group.
auc = np.zeros(n_samp)
for m in range(n_pred-1):
    auc += logistic(Beta_mu[:,:m+1].sum(axis=-1) + np.outer(Beta_mu[:,-1], reward)).mean(axis=1)
auc /= n_pred - 1.   

## Merge.
AUC = np.concatenate([AUC,auc[:,np.newaxis]], axis=-1)
subjects = np.append(results['Subjects'], 'Group')
n_subj = subjects.shape[0]

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Plotting likelihood of take (AUC).
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Re-sort data by median.
ix = np.argsort(np.median(AUC, axis=0))
AUC = AUC[:,ix]
subjects = subjects[ix]

## Mask data outside HDI.
mask = np.apply_along_axis(hdi_mask, 0, AUC, hdi)

## Initialize plot
fig = plt.figure(figsize=(15,10))
bins = np.linspace(0,1,51)
pal = sns.cubehelix_palette(n_subj, start=0.2, rot=-0.5, dark=0.25, light=.7)

for n in range(n_subj):
    
    ## Open ax. 
    ax = plt.subplot2grid((5,6),(n/6,n%6))
    
    ## Plot histogram.
    ax.hist(mask.T[n][~np.isnan(mask.T[n])], bins, histtype='bar', facecolor=pal[n], lw=0.1)
    ax.set_xticks(np.linspace(0,1,5))
    ax.set_yticks([])
    ax.tick_params(axis='x', which='major', labelsize=12)
    if subjects[n].startswith('hc'): ax.set_title(subjects[n].upper(), fontsize=14, alpha=0.6)
    else: ax.set_title('Group', fontsize=18, weight='bold')
    
    ## Turn off axes.
    ax.spines['left'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    
plt.tight_layout()
# plt.show()
plt.savefig('plots/FINAL/behavior_likelihood.png',dpi=120)
plt.close()

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Plotting RT slowdown to conflict.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Merge.
a1 = np.concatenate([results['a1'],results['a1_mu'][:,np.newaxis]], axis=-1)
a1 /= 4. # To get accurate RT effect.

subjects = np.append(results['Subjects'], 'Group')
n_subj = subjects.shape[0]

## Re-sort data by median.
ix = np.argsort(np.median(a1, axis=0))
a1 = a1[:,ix]
subjects = subjects[ix]

## Mask data outside HDI.
mask = np.apply_along_axis(hdi_mask, 0, a1, hdi)

## Initialize plot
fig = plt.figure(figsize=(15,10))
bins = np.linspace(0,1,51)
pal = sns.cubehelix_palette(n_subj, start=0.2, rot=-0.5, dark=0.25, light=.7)

for n in range(n_subj):
    
    ## Open ax. 
    ax = plt.subplot2grid((5,6),(n/6,n%6))
    
    ## Plot histogram.
    ax.hist(mask.T[n][~np.isnan(mask.T[n])], bins, color=pal[n], lw=0.1)
    ax.set_xticks(np.linspace(0,1,5))
    ax.set_yticks([])
    ax.tick_params(axis='x', which='major', labelsize=12)
    if subjects[n].startswith('hc'): ax.set_title(subjects[n].upper(), fontsize=14, alpha=0.6)
    else: ax.set_title('Group', fontsize=18, weight='bold')
    
    ## Turn off axes.
    ax.spines['left'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    
plt.tight_layout()
# plt.show()
plt.savefig('plots/FINAL/behavior_rt_conflict.png',dpi=120)
plt.close()

# Behavior Violin Plots

In [64]:
import cPickle, os
import numpy as np
import pylab as plt
import seaborn as sns
from pandas import DataFrame
sns.set_style(style='white')
sns.set_context("paper", font_scale=2)
plt.ion()

def logistic(arr): return 1. / (1 + np.exp(-arr))

def HDIofMCMC(arr, credMass=0.95):
    '''
    Computes highest density interval from a sample of representative values,
    estimated as shortest credible interval. Functions for computing HDI's are 
    explained in Chapter 25 of Doing Bayesian Data Analysis, Second Edition.
    
    INPUTS:
    -- arr: a vector of representative values from a probability distribution.
    -- credMass: a scalar between 0 and 1, indicating the mass within the credible
       interval that is to be estimated.
    '''
    sortedPts = np.sort(arr)
    ciIdxInc = np.ceil(credMass * len( sortedPts )).astype(int)
    nCIs = len( sortedPts ) - ciIdxInc
    ciWidth = [ sortedPts[ i + ciIdxInc ] - sortedPts[ i ] for i in np.arange(nCIs).astype(int) ]
    HDImin = sortedPts[ np.argmin( ciWidth ) ]
    HDImax = sortedPts[ np.argmin( ciWidth ) + ciIdxInc ]
    return HDImin, HDImax

def hdi_mask(arr,p):
    lb, ub = HDIofMCMC(arr, p)
    return np.where(arr < lb, np.nan, np.where(arr > ub, np.nan, arr))

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## I/O parameters.
results_file = 'arc_hierarchical_add_FINAL'

## Sampling parameters.
n_reward = 1000
hdi = 0.95

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Load data.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Open results file.
print 'Loading data.'
f = 'stan_results/%s.pickle' %results_file
with open(f, 'rb') as f: results = cPickle.load(f)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Re-standardize coefficients
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
## See Kruschke, 2011 [pgs. 424, 566]

## Make a copy.
X = results['X']
Beta = results['beta'].copy()
Beta_mu = results['beta_mu'].copy()

## Transform the intercept coefficient.
Beta[:,:,0] = Beta[:,:,0] - (Beta[:,:,-1] * np.mean(X[:,-1]) / np.std(X[:,-1]))    
Beta_mu[:,0] = Beta_mu[:,0] - (Beta_mu[:,-1] * np.mean(X[:,-1]) / np.std(X[:,-1]))    

## Transform the reward coefficient.
Beta[:,:,-1] = Beta[:,:,-1] / np.std(X[:,-1]) 
Beta_mu[:,-1] = Beta_mu[:,-1] / np.std(X[:,-1]) 

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Compute AUC.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
print 'Computing AUC.'

## Get subjects list. Preallocate space.
n_samp, n_subj, n_pred = Beta.shape
AUC = np.zeros((n_samp, n_subj))

## Compute AUC across subjects.
reward = np.linspace(0,1,n_reward)
for n in range(n_subj):
    
    for m in range(n_pred-1):
        
        AUC[:,n] += logistic(Beta[:,n,:m+1].sum(axis=-1) + np.outer(Beta[:,n,-1], reward)).mean(axis=1)
        
AUC /= n_pred - 1.

## Compute AUC for group.
auc = np.zeros(n_samp)
for m in range(n_pred-1):
    auc += logistic(Beta_mu[:,:m+1].sum(axis=-1) + np.outer(Beta_mu[:,-1], reward)).mean(axis=1)
auc /= n_pred - 1.   

## Delete variables (unnecessary).
del Beta, Beta_mu

## Mask data outside HDI.
AUC = np.apply_along_axis(hdi_mask, 0, AUC, hdi)
auc = hdi_mask(auc, hdi)

## Merge data.
AUC = np.concatenate([AUC,auc[:,np.newaxis]],axis=-1)
subjects = np.append(results['Subjects'], 'Group')

## Re-sort by median AUC.
ix = np.argsort(np.nanmedian(AUC,0))
AUC = AUC[:,ix]
subjects = subjects[ix]

## Prepare subjects data.
AUC = AUC.T.flatten()
subjects = np.concatenate([np.repeat(subject,n_reward) for subject in subjects])

## Place into DataFrame.
df = DataFrame(np.vstack([AUC,subjects]).T, columns=('AUC','Subjects'))
df['AUC'] = df['AUC'].astype(float)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Plotting likelihood of take (AUC).
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Generate colors.
colors = ['#7ec0ee' if s.startswith('hc') else '#74c476' for s in df.Subjects.unique()]
palette = sns.color_palette(colors)

## Generate violinplot.
fig, ax = plt.subplots(1,1,figsize=(12,4))
sns.violinplot(x='Subjects', y='AUC',data=df, cut=0, scale='count', width=0.7,
               linewidth=0.1, palette=palette, saturation=1, scale_hue=False, ax=ax)

## Add horizontal lines.
xmin, xmax = ax.get_xlim()
sns.plt.hlines(df.loc[df.Subjects=='Group','AUC'].median(), xmin, xmax,
          linestyle='--', linewidth=0.75, alpha=0.5, zorder=-1)

## Add legend.
ax.plot([],[],color='#74c476',label='Group')
ax.plot([],[],color='#7ec0ee',label='Subject')
legend = ax.legend(loc=4, frameon=False, borderaxespad=0, handletextpad=0.5,
                  borderpad=0.5, handlelength=1.75)
for l in legend.get_lines(): 
    l.set_linewidth(10)
    l.set_solid_capstyle('butt')

## Fix axes.
# ax.set_xticklabels(['s%s' %s[-2:] if s.startswith('hc') else 'G' for s in df.Subject.unique()])
ax.set_xticklabels([])
ax.set_ylim(0,1)
ax.set_ylabel(r'Average Likelihood-of-Take ($\theta$)')

## Save.
sns.despine()
plt.tight_layout()
# plt.show()
plt.savefig('plots/FINAL/behavior_likelihood_violin.png',dpi=120)
plt.close()

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Prepare RT-Conflict Slope Data.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Extract and merge.
a1 = np.concatenate([results['a1'],results['a1_mu'][:,np.newaxis]], axis=-1)
a1 /= 4. # To get accurate RT effect.

subjects = np.append(results['Subjects'], 'Group')
n_subj = subjects.shape[0]

## Re-sort data by median.
ix = np.argsort(np.median(a1, axis=0))
a1 = a1[:,ix]
subjects = subjects[ix]

## Mask data outside HDI.
a1 = np.apply_along_axis(hdi_mask, 0, a1, hdi)

## Prepare subjects data.
a1 = a1.T.flatten()
subjects = np.concatenate([np.repeat(subject,n_reward) for subject in subjects])

## Place into DataFrame.
df = DataFrame(np.vstack([a1,subjects]).T, columns=('a1','Subjects'))
df['a1'] = df['a1'].astype(float)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Plotting likelihood of take (AUC).
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Generate colors.
colors = ['#7ec0ee' if s.startswith('hc') else '#74c476' for s in df.Subjects.unique()]
palette = sns.color_palette(colors)

## Generate violinplot.
fig, ax = plt.subplots(1,1,figsize=(12,4))
sns.violinplot(x='Subjects', y='a1',data=df, cut=0, scale='count', width=0.7,
               linewidth=0.1, palette=palette, saturation=1, scale_hue=False, ax=ax)

## Add horizontal lines.
xmin, xmax = ax.get_xlim()
sns.plt.hlines(df.loc[df.Subjects=='Group','a1'].median(), xmin, xmax,
          linestyle='--', linewidth=0.75, alpha=0.5, zorder=-1)

## Add legend.
ax.plot([],[],color='#74c476',label='Group')
ax.plot([],[],color='#7ec0ee',label='Subject')
legend = ax.legend(loc=4, frameon=False, borderaxespad=0, handletextpad=0.5,
                  borderpad=0.5, handlelength=1.75)
for l in legend.get_lines(): 
    l.set_linewidth(10)
    l.set_solid_capstyle('butt')

## Fix axes.
# ax.set_xticklabels(['s%s' %s[-2:] if s.startswith('hc') else 'G' for s in df.Subject.unique()])
ax.set_xticklabels([])
ax.set_ylim(0,1)
ax.set_ylabel(r'Conflict-RT Slope ($\alpha_1$)')

## Save.
sns.despine()
plt.tight_layout()
# plt.show()
plt.savefig('plots/FINAL/behavior_rt_conflict_violin.png',dpi=120)
plt.close()
print 'Done.'

Loading data.
Computing AUC.
Done.


# Mindboggle Parcellation

In [ ]:
import numpy as np
from mne import read_label
from surfer import Brain
from seaborn import color_palette
%matplotlib qt4

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## I/O parameters.
fs_dir = '/space/lilli/1/users/DARPA-Recons'
subject = 'fsaverage'
surface = 'inflated'
hemi = 'lh'

## Label paramaters.
label_dir = '/space/lilli/1/users/DARPA-Recons/fscopy/label/dkt40'
roi_list = ['caudalanteriorcingulate', 'rostralanteriorcingulate', 'posteriorcingulate',
            'superiorfrontal', 'medialorbitofrontal', 'rostralmiddlefrontal', 'caudalmiddlefrontal',
            'parsopercularis', 'parstriangularis', 'parsorbitalis', 'lateralorbitofrontal', 'insula']

colors = [[0.78329874347238, 0.687243385525311, 0.8336793640080622],
          [0.1257208769520124, 0.47323337360924367, 0.707327968232772],
            [0.999907727802501, 0.5009919264737298, 0.005121107311809869],
          [0.21171857311445125, 0.6332641510402455, 0.1812226118410335],
          [0.6941176652908325, 0.3490196168422699, 0.1568627506494522],
          [0.42485198495434734, 0.2511495584950722, 0.6038600774372326],
          [0.983206460055183, 0.5980161709820524, 0.5942330108845937],
          [0.9917570170234231, 0.7464821371669862, 0.4340176893507733],
          [0.8905959311653586, 0.10449827132271793, 0.111080354627441],
          [0.9976009228650261, 0.9948942715046452, 0.5965244373854468],
          [0.6889965575115352, 0.8681737867056154, 0.5437601194662207],
          [0.6509804129600525, 0.8078431487083435, 0.8901960849761963]]

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Visualize brain.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Load labels.
roi_list = [read_label('%s/%s-%s.label' %(label_dir,roi,hemi), subject=subject) for roi in roi_list]

## Make brain.
brain = Brain(subject, hemi, surface, background='white', subjects_dir=fs_dir)
for roi, color in zip(roi_list, colors): brain.add_label(roi, color=color)
for view in ['medial','lateral']:
    brain.show_view(view)
    brain.save_image('plots/FINAL/mindboggle_%s.png' %view)

# 6-panel Deliberation Map

In [ ]:
import os 
import numpy as np
from surfer import Brain
%matplotlib qt4

fs_dir = '/space/lilli/1/users/DARPA-Recons'
img_dir = 'plots/FINAL'

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#


## I/O parameters.
sm = 6
fd = 0.9
contrast = 'Delib'
overlay = 'psc'

## Pysurfer parameters.
surface = 'inflated'
views = ['lateral','medial']
size = (1200,800)
bg_color = 'black'

fmin = 0.04 # min value
fmax = 0.20 # max value

## Label parameters.
label_dir = 'fmri_second_levels/labels/seeds_%s' %contrast
labels = ['dacc', 'dlpfc', 'insula', 'mcc', 'pre_sma']
label_color = 'k'

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Plot.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Initialize brain.
brains = Brain("fsaverage", 'split', surface, views=views, size=size,
              background=bg_color, subjects_dir=fs_dir)

## Add overlay.
for hemi in ['lh','rh']:
    f = os.path.join('fmri_second_levels', 'FINAL.%s.%s.%s.%s' %(sm, fd, hemi, contrast), '%s.nii.gz' %overlay)
    brains.add_overlay(f, min=fmin, max=fmax, sign="pos", hemi=hemi)
    
## Add labels.
# for label in labels:
#     for hemi in ['lh','rh']:
#         f = os.path.join(label_dir, 'fig_%s-%s.label' %(label,hemi))
#         brains.add_label(f, color=label_color, borders=True, hemi=hemi)
    
## Update views.
brains.set_distance(300)
brains.hide_colorbar(0,0)
brains.hide_colorbar()

brains.save_image('plots/FINAL/delib.png')

# 3-panel Delibmod

In [ ]:
import os, time
import numpy as np
from surfer import Brain
%matplotlib qt4

fs_dir = '/space/lilli/1/users/DARPA-Recons'
img_dir = 'plots/FINAL'

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## I/O parameters.
sm = 6
fd = 0.9
contrast = 'DelibMod'
overlay = 'psc'

## Pysurfer parameters.
surface = 'pial'
size = (800,400)
bg_color = 'black'

fmin = 1e-6 # min value
fmax = 0.05 # max value

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Plot.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

hemis = ['lh','rh','rh']
views = ['lateral','lateral','medial']
params = [dict(azimuth=150, roll=90),dict(azimuth=30, roll=270),dict(azimuth=180, roll=90)]
distances = [280,280,280]

for hemi, view, param, dist in zip(hemis,views,params,distances):

    ## Initialize brain.
    brain = Brain("fsaverage", hemi, surface, views=[view], size=size,
                  background=bg_color, subjects_dir=fs_dir)

    ## Add overlay.
    f = os.path.join('fmri_second_levels', 'FINAL.%s.%s.%s.%s' %(sm, fd, hemi, contrast), '%s.nii.gz' %overlay)
    brain.add_overlay(f, min=fmin, max=fmax, sign="pos", hemi=hemi)
    
    ## Update views.
    brain.show_view(param)
    brain.set_distance(dist)
    brain.hide_colorbar()
    time.sleep(5)
    brain.save_image('plots/FINAL/delibmod_%s_%s.png' %(hemi,view))
    
print 'Done.'

# Pysurfer Colorbar
Convenience

In [ ]:
import os 
import numpy as np
from surfer import Brain
%matplotlib qt4

fs_dir = '/space/lilli/1/users/DARPA-Recons'
img_dir = 'plots/FINAL'

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Pysurfer parameters.
size = (1200,800)
bg_color = 'black'

fmin = 0.00 # min value
fmax = 0.05 # max value

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Plot.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Initialize brain.
brains = Brain("fsaverage", 'lh', 'inflated', size=size,
              background=bg_color, subjects_dir=fs_dir)

## Add overlay.    
f = 'fmri_second_levels/FINAL.6.0.9.lh.Delib/psc.nii.gz'
brains.add_overlay(f, min=fmin, max=fmax, sign="pos")

In [ ]:
brains.data_dict